In [1]:
import os
import sys
sys.path.append(os.path.dirname(os.path.realpath('__file__')) + '/../../../')
from acaisdk.file import File
from acaisdk.project import Project
from acaisdk.fileset import FileSet
from acaisdk.job import Job
from acaisdk.meta import *
from acaisdk.utils import utils
from acaisdk import credentials
# from acaisdk import automl

utils.DEBUG = True  # print debug messages
workspace = os.path.dirname(os.path.realpath('__file__'))  # get current directory

In [2]:
# Setting up project
# 
# Either:
# 1) Provide your existing token for some project
# 2) Fill in information for new project creation (project_name, project_admin, project_user, optional [csp, budget])
# Do not change root_token!
token = 'Uj5DkOSYMPgvPsbNcs6DIEjh8eE2ooTU'
force = False
    
project_name = ''
project_admin = 'admin_1'
project_user = 'hangruic'
csp = 'AWS' # AWS/AZURE/GCP/PRIVATE
budget = 10 # Default, I have no idea what other options there are

try:
    p
except NameError:
    pass
else:
    if not force:
        if token == '':
            print("User token {} already exists, saving to variable 'token'. If you want to enforce new project, set 'force=true'.".format(p['user_token']))
            token = p['user_token']
    else:
        print("User token {} already exists but forcing new project.".format(p['user_token']))


if token != '':
    print("Logging in with existing credentials.")
    credentials.login(token)
else:
    print("Creating new project.")
    if project_name == '' or project_admin == '' or project_user == '':
        raise ValueError("Some of the 'project_name', 'project_admin', 'project_user' not provided!")
    
    root_token = 'EmDlCTBF1ppONSciYVd03M9xkmF6hFqW' 
    p = Project.create_project(project_name, root_token, project_admin, csp=csp, budget=budget)
    p = Project.create_user(project_name, p['project_admin_token'], project_user)
    token = p['user_token']
    print("New user token {}, saved to variable 'token'".format(token))

Logging in with existing credentials.


In [ ]:
# Set env variable needed for Phoebe
USE_PHOEBE = False
if USE_PHOEBE:
    os.environ["CLUSTER"] = 'PHOEBE'

In [4]:
File.list_dir('/')

Running request: 104.196.115.179 80 storage list_directory
GET query {"directory_path": "/", "token": "Uj5DkOSYMPgvPsbNcs6DIEjh8eE2ooTU"}


ConnectionError: HTTPConnectionPool(host='104.196.115.179', port=80): Max retries exceeded with url: /storage/list_directory?directory_path=%2F&token=Uj5DkOSYMPgvPsbNcs6DIEjh8eE2ooTU (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fab7cf49610>: Failed to establish a new connection: [Errno 60] Operation timed out'))

In [ ]:
outputs = os.path.join(workspace, 'outputs/')
if not os.path.isdir(outputs):
    os.mkdir(outputs)
inputs = os.path.join(workspace, 'inputs/')
if not os.path.isdir(inputs):
    os.mkdir(inputs)
inf = [f for f in os.listdir(inputs) if os.path.isfile(os.path.join(inputs, f))]
print(inf)

if inf == []:
    raise FileNotFoundError("Data is not extracted in the inputs directory! Extract the data NLP-GloVe/fake-and-real-news-dataset.zip NLP-GloVe/glove-twitter.zip to inputs directory (e.g. tar -xvf glove-twitter.zip -C inputs)!")

In [ ]:
File.convert_to_file_mapping([inputs], '/cnn-inputs/') \
    .files_to_upload \
    .upload() \
    .as_new_file_set('cnn.inputs')

In [ ]:
File.list_dir('cnn-inputs/')

In [ ]:
# Upload code
code = os.path.join(workspace, 'cnn.zip')
File.upload({code: 'cnn.zip'})